# Example: Relaying Jobs via `relay_server.py`

This notebook demonstrates how to submit a job through the `relay_server.py` to a backend `server_app.py` (reconstruction server).

**Functionality Demonstrated:**
*   Client submits a job to the Relay Server.
*   Relay Server receives the job and its files, stores files temporarily.
*   Relay Server selects a Backend Server.
*   Relay Server forwards the job and files to the Backend Server.
*   Backend Server queues the job.
*   Relay Server confirms to the Client that the job has been relayed and queued on a backend.

**Prerequisites:**
1.  The core job forwarding logic (client -> relay -> backend queueing) must be implemented and working in `relay_server.py`.
2.  Full result relaying (DICOMs and final status from backend -> relay -> client) might be a pending feature in `relay_server.py`. This notebook primarily focuses on the submission path.

## 1. Setup Instructions

To run this notebook successfully, you need three components running, ideally in separate terminals:

### 1.1. Backend Reconstruction Server (`server_app.py`)

*   **Configuration (`recon.opts` for the backend server):**
    *   Ensure `SERVER_PORT` is set (e.g., `SERVER_PORT = 60000`).
    *   Set a unique `SHARED_KEY` (e.g., `SHARED_KEY = YourBackendServerKey`). This key will be used by the Relay to connect to this Backend.
    *   Optionally, configure `SERVER_ADMIN_PORT` and `SERVER_ADMIN_SHARED_KEY` if you want to monitor it with `server_admin_cli.py`.
    *   Make sure `RECON_SCRIPT_PATH` points to a valid (even if dummy) executable script. For this test, a simple script that creates a dummy output file in the job's output directory would suffice if you want to test end-to-end file creation.
    ```
    # Example recon.opts for backend server_app.py
    SERVER_HOSTNAME = localhost
    SERVER_PORT = 60000 
    SHARED_KEY = YourBackendServerKey # IMPORTANT! Used by Relay to connect to this backend
    LOG_FILEPATH = /tmp/backend_server.log
    LOG_LEVEL = DEBUG 
    RECON_SERVER_BASE_PFILE_DIR = /tmp/backend_server_pfiles
    RECON_JOB_OUTPUT_BASE_DIR = /tmp/backend_server_job_outputs
    RECON_SCRIPT_PATH = ./default_recon_script.sh # Ensure this exists and is executable
    MAX_CONCURRENT_JOBS = 1
    SERVER_ADMIN_PORT = 60003
    SERVER_ADMIN_SHARED_KEY = YourBackendAdminKey
    MAX_CPU_LOAD_PERCENT = 90
    MIN_AVAILABLE_MEMORY_MB = 100
    ```
*   **Command to Run:**
    ```bash
    python server_app.py --opts recon.opts # (ensure recon.opts is configured for backend)
    ```

### 1.2. Relay Server (`relay_server.py`)

*   **Configuration (`relay.opts` for the relay server):**
    *   `RELAY_PORT`: Port for clients (this notebook) to connect to (e.g., `RELAY_PORT = 60001`).
    *   `SHARED_KEY_RELAY_TO_CLIENTS`: Shared key for clients connecting to the relay (e.g., `SHARED_KEY_RELAY_TO_CLIENTS = YourRelayToClientKey`).
    *   `BACKEND_SERVERS`: List of backend server addresses. For this test, it should point to your running `server_app.py` instance (e.g., `BACKEND_SERVERS = localhost:60000`).
    *   `SHARED_KEY_FOR_BACKENDS`: **Must match** the `SHARED_KEY` of the backend `server_app.py` (e.g., `SHARED_KEY_FOR_BACKENDS = YourBackendServerKey`).
    *   `RELAY_JOB_TEMP_DIR`: Path for relay's temporary file storage (e.g., `/tmp/relay_job_files`).
    *   Optionally, configure `RELAY_ADMIN_PORT` and `RELAY_ADMIN_SHARED_KEY` for `relay_server_cli.py`.
    ```
    # Example relay.opts
    RELAY_HOSTNAME = localhost
    RELAY_PORT = 60001
    SHARED_KEY_RELAY_TO_CLIENTS = YourRelayToClientKey # For this notebook to connect to relay
    LOG_FILEPATH = /tmp/relay_server.log
    LOG_LEVEL = DEBUG
    BACKEND_SERVERS = localhost:60000 # Points to server_app.py instance
    SHARED_KEY_FOR_BACKENDS = YourBackendServerKey # MUST MATCH SHARED_KEY in backend's recon.opts
    RELAY_JOB_TEMP_DIR = /tmp/relay_job_files
    RELAY_ADMIN_PORT = 60002
    RELAY_ADMIN_SHARED_KEY = YourRelayAdminKey
    ```
*   **Command to Run:**
    ```bash
    python relay_server.py --opts relay.opts
    ```

### 1.3. Client (This Notebook)

*   This notebook will act as the client.
*   We will create a temporary configuration file (`client_to_relay.opts`) for the `ReconClientApp` instance used by this notebook. This configuration will point to the **Relay Server's client-facing port** and use the **`SHARED_KEY_RELAY_TO_CLIENTS`**.

### 1.4. Dummy Input Files

We'll create some dummy files for the job submission.

In [ ]:
import os
import json

dummy_files_dir = "dummy_input_files_for_relay_test"
os.makedirs(dummy_files_dir, exist_ok=True)

dummy_file1_path = os.path.join(dummy_files_dir, "P_dummy_relay1.7")
dummy_file2_path = os.path.join(dummy_files_dir, "calibration_relay_dummy.dat")

with open(dummy_file1_path, 'w') as f:
    f.write("This is a dummy PFILE for relay testing.")

with open(dummy_file2_path, 'w') as f:
    f.write("This is a dummy calibration file for relay testing.")

print(f"Created dummy file: {dummy_file1_path}")
print(f"Created dummy file: {dummy_file2_path}")

recon_options_example = {
    "pyscript_name": "slicerecon_ relayed",
    "num_slices": 64,
    "relayed_job": True
}
print(f"Example recon options for relayed job: {json.dumps(recon_options_example)}")

## 2. Client Configuration and Initialization

We need to configure our `ReconClientApp` instance to talk to the **Relay Server**.

In [ ]:
import sys
project_root = os.path.abspath("..") # If notebook is in a subdir, adjust if needed
if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    from client_app import ReconClientApp
    from reconlibs import readoptions # readoptions is used by client_app
    print("Successfully imported ReconClientApp and readoptions.")
except ImportError as e:
    print(f"Error importing modules: {e}. Ensure this notebook is in the project root or adjust sys.path.")
    # You might need to restart the kernel after fixing the path.

# Create a temporary opts file for this notebook's client instance
client_opts_content = """
SERVER_HOSTNAME = localhost
SERVER_PORT = 60001 # Relay's client-facing port
SHARED_KEY = YourRelayToClientKey # Key for client to relay communication
LOG_FILEPATH = /tmp/notebook_client_to_relay.log
LOG_LEVEL = DEBUG
CLIENT_DOWNLOAD_DIR = client_downloads_from_relay_test
"""
notebook_client_opts_file = "temp_client_to_relay.opts"
with open(notebook_client_opts_file, 'w') as f:
    f.write(client_opts_content)
print(f"Created temporary client opts file: {notebook_client_opts_file}")

# Initialize ReconClientApp
relay_client = ReconClientApp(options_file=notebook_client_opts_file)
print("ReconClientApp (to connect to Relay) initialized.")

## 3. Submitting Job to Relay Server

In [ ]:
job_submission_response = None

if relay_client.connect():
    print("Connected to Relay Server successfully.")
    
    files_to_submit = [dummy_file1_path, dummy_file2_path]
    
    # The submit_recon_job method handles the interaction for submitting the job and its files
    # and waits for the 'job_queued' (or in this case 'job_relayed_and_queued') response.
    relayed_job_id = relay_client.submit_recon_job(
        files_to_process=files_to_submit, 
        recon_options=recon_options_example
    )
    
    if relayed_job_id:
        print(f"Job successfully submitted to Relay. Relay Job ID (from client perspective): {relayed_job_id}")
        # The actual response from relay_server's 'job_relayed_and_queued' might be more complex.
        # We need to inspect what submit_recon_job actually returns in the context of a relay.
        # For now, we assume `relayed_job_id` is the `job_id` from the *backend* if the relay passes it through,
        # or the relay's own job ID.
        # The `submit_recon_job` in `client_app` expects the `job_id` in the 'job_queued' payload.
        # Relay server sends 'job_relayed_and_queued' with 'relay_job_id' and 'backend_job_id'.
        # The client's submit_recon_job was designed for 'job_queued', so its direct return might be the 'job_id' field if present.
        # We should check the actual last message or adapt client if needed.
        
        # Let's assume the `submit_recon_job` returns the primary job ID it cares about (which would be the backend one if the relay is transparent about it)
        # or we can inspect the last message if the relay sends a custom response type.
        last_message = relay_client.sf_client.last_received_message
        if last_message and last_message.get("type") == "job_relayed_and_queued":
            print("Detailed response from Relay:")
            print(json.dumps(last_message.get("payload"), indent=2))
            job_submission_response = last_message.get("payload")
        else:
            print(f"Standard job_id from submit_recon_job: {relayed_job_id} (may or may not be backend_job_id depending on relay's response structure)")
            print(f"Last raw message from server (relay): {last_message}")
            
    else:
        print("Job submission to Relay failed.")
    
    # Note: The client's default `process_job_and_get_results` would expect DICOMs etc.
    # from the direct server it connected to (the relay). This part of the relay is TBD.
    
    relay_client.disconnect()
    print("Disconnected from Relay Server.")
else:
    print("Failed to connect to the Relay Server. Ensure it's running and client_to_relay.opts is correct.")

## 4. Monitoring the System (Using Admin CLIs)

After submitting the job to the relay, you can use the provided CLI tools in separate terminals to monitor the components.

**Assumptions for CLI commands:**
*   You have `relay.opts` configured for the relay server (including its admin port/key).
*   You have `recon.opts` configured for the backend `server_app.py` (including its admin port/key).

### 4.1. Check Relay Server Status
```bash
# In a new terminal:
python relay_server_cli.py status --opts relay.opts 
```
*(Expected: Shows relay uptime, client connection count during submission, etc.)*

### 4.2. Check Relay's Backend Health
```bash
# In a new terminal:
python relay_server_cli.py backends --opts relay.opts
```
*(Expected: Shows the backend server (e.g., `localhost:60000`) and its health status as 'Yes' if the relay can connect to it.)*

### 4.3. Check Backend Server's Job Queue
```bash
# In a new terminal, targeting the backend server's recon.opts:
python server_admin_cli.py queue --opts recon.opts 
```
*(Expected: You should see the job (identified by `backend_job_id` from the relay's response) listed in the backend server's queue.)*

### 4.4. Check Backend Server's Worker Status
```bash
# In a new terminal, targeting the backend server's recon.opts:
python server_admin_cli.py workers --opts recon.opts
```
*(Expected: If the job is simple/fast, it might already be processing or completed. You might see a worker busy with the `backend_job_id`.)*

### Optional: Running CLI commands from Notebook (for quick check)

In [ ]:
import subprocess

def run_cli_command(command):
    print(f"\nRunning: {command}\n")
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        print("Output:\n", result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error Output:\n", e.stderr)
    except FileNotFoundError:
        print(f"Error: Command not found. Ensure the CLI script is in the path or project root.")

# Note: These commands assume relay.opts and recon.opts are in the same directory as the notebook.
# You might need to adjust paths or run servers first.

print("--- Checking Relay Status (relay.opts) ---")
run_cli_command("python relay_server_cli.py status --opts relay.opts")

print("\n--- Checking Backend Health via Relay (relay.opts) ---")
run_cli_command("python relay_server_cli.py backends --opts relay.opts")

print("\n--- Checking Backend Server Queue (recon.opts for backend) ---")
run_cli_command("python server_admin_cli.py queue --opts recon.opts")

## 5. Result Handling

The current implementation of `relay_server.py` focuses on forwarding the job submission to a backend server and confirming to the client that the job has been *queued* on the backend. 

**Key points regarding results:**
*   **Job Queued Confirmation**: The notebook demonstrates receiving the `job_relayed_and_queued` message, which includes the `relay_job_id` (internal to relay operations) and the `backend_job_id` (the ID of the job on the actual reconstruction server).
*   **Result File Relaying (TBD)**: The full relaying of result files (e.g., DICOMs) and final status messages (`recon_complete` or `job_failed`) from the backend server, through the relay, and back to the original client is a more complex part of the relay functionality and is **considered a pending feature for `relay_server.py`** as of this notebook's creation.
*   **Client Behavior**: The `ReconClientApp`'s `submit_recon_job` method as used here is primarily for the submission phase. A more complete interaction involving result download would typically use `run_full_job_cycle` or a sequence including `process_job_and_get_results(job_id)`. However, these methods in the client would expect the *relay server* to behave like the *end server* regarding result delivery. This requires the relay to implement the result forwarding logic.

To get actual results for a job submitted via the relay, you would currently need to interact directly with the backend server, potentially using its `CLIENT_DOWNLOAD_DIR` if you have access, or by implementing a client that connects directly to the backend server once you know the `backend_job_id`.

## 6. Cleanup

In [ ]:
print("Cleanup: Remember to stop the backend server_app.py and relay_server.py processes in their terminals.")

try:
    if os.path.exists(notebook_client_opts_file):
        os.remove(notebook_client_opts_file)
        print(f"Removed temporary client opts file: {notebook_client_opts_file}")
    
    if os.path.exists(dummy_files_dir):
        shutil.rmtree(dummy_files_dir)
        print(f"Removed dummy input files directory: {dummy_files_dir}")
except Exception as e:
    print(f"Error during cleanup: {e}")